# MyScale

>[MyScale](https://docs.myscale.com/en/) 是一个集成的数据库。你可以通过 SQL 和 LangChain 访问你的数据库。
>`MyScale` 可以利用[各种数据类型和函数进行过滤](https://blog.myscale.com/2023/06/06/why-integrated-database-solution-can-boost-your-llm-apps/#filter-on-anything-without-constraints)。无论你是扩展数据还是将系统扩展到更广泛的应用，它都能提升你的 LLM 应用。

在本 notebook 中，我们将演示围绕 `MyScale` 向量存储包装的 `SelfQueryRetriever`，以及我们为 LangChain 贡献的一些额外功能。

简而言之，可以归纳为 4 点：
1. 添加 `contain` 比较器，以匹配包含一个以上元素的列表
2. 添加 `timestamp` 数据类型以进行日期时间匹配（ISO 格式或 YYYY-MM-DD）
3. 添加 `like` 比较器以进行字符串模式搜索
4. 添加任意函数功能

## 创建 MyScale 向量存储
MyScale 已集成到 LangChain 中一段时间了。因此，您可以按照[本笔记本](/docs/integrations/vectorstores/myscale)创建自己的向量存储以用于自查询检索器。

**注意：**所有自查询检索器都需要安装 `lark` (`pip install lark`)。我们使用 `lark` 进行语法定义。在进行下一步之前，我们还要提醒您，还需要 `clickhouse-connect` 来与您的 MyScale 后端进行交互。

In [ ]:
%pip install --upgrade --quiet  lark clickhouse-connect

在本教程中，我们遵循其他示例的设置并使用 `OpenAIEmbeddings`。请记住，您需要获取一个 OpenAI API 密钥才能有效访问 LLMs。

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
if "MYSCALE_HOST" not in os.environ:
    os.environ["MYSCALE_HOST"] = getpass.getpass("MyScale URL:")
if "MYSCALE_PORT" not in os.environ:
    os.environ["MYSCALE_PORT"] = getpass.getpass("MyScale Port:")
if "MYSCALE_USERNAME" not in os.environ:
    os.environ["MYSCALE_USERNAME"] = getpass.getpass("MyScale Username:")
if "MYSCALE_PASSWORD" not in os.environ:
    os.environ["MYSCALE_PASSWORD"] = getpass.getpass("MyScale Password:")

In [ ]:
from langchain_community.vectorstores import MyScale
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## 创建一些示例数据
正如你所见，我们创建的数据与其他的 self-query retrievers 相比有一些不同之处。我们将关键字 `year` 替换为 `date`，这让你对时间戳有更精细化的控制。我们还将关键字 `gerne` 的类型更改为字符串列表，在这里，LLM 可以使用新的 `contain` 比较器来构建过滤器。我们还为过滤器提供了 `like` 比较器和任意函数支持，这些将在接下来的几个单元格中介绍。

现在让我们先看看数据。

In [ ]:
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"date": "1993-07-02", "rating": 7.7, "genre": ["science fiction"]},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"date": "2010-12-30", "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"date": "2006-04-23", "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"date": "2019-08-22", "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"date": "1995-02-11", "genre": ["animated"]},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "date": "1979-09-10",
            "director": "Andrei Tarkovsky",
            "genre": ["science fiction", "adventure"],
            "rating": 9.9,
        },
    ),
]
vectorstore = MyScale.from_documents(
    docs,
    embeddings,
)

## 创建我们的自查询检索器
就像其他检索器一样……简单而nice。

In [ ]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genres of the movie. "
        "It only supports equal and contain comparisons. "
        "Here are some examples: genre = [' A '], genre = [' A ', 'B'], contain (genre, 'A')",
        type="list[string]",
    ),
    # If you want to include length of a list, just define it as a new column
    # This will teach the LLM to use it as a column when constructing filter.
    AttributeInfo(
        name="length(genre)",
        description="The length of genres of the movie",
        type="integer",
    ),
    # Now you can define a column as timestamp. By simply set the type to timestamp.
    AttributeInfo(
        name="date",
        description="The date the movie was released",
        type="timestamp",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]
document_content_description = "Brief summary of a movie"
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

## 使用自查询检索器的现有功能进行测试
现在我们可以尝试实际使用我们的检索器了！

In [ ]:
# This example only specifies a relevant query
retriever.invoke("What are some movies about dinosaurs")

In [ ]:
# This example only specifies a filter
retriever.invoke("I want to watch a movie rated higher than 8.5")

In [ ]:
# This example specifies a query and a filter
retriever.invoke("Has Greta Gerwig directed any movies about women")

In [ ]:
# This example specifies a composite filter
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

In [ ]:
# This example specifies a query and composite filter
retriever.invoke(
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

# 等等……还有什么？

带有 MyScale 的自查询检索器还能做更多！让我们一起来看看吧。

In [ ]:
# You can use length(genres) to do anything you want
retriever.invoke("What's a movie that have more than 1 genres?")

In [ ]:
# Fine-grained datetime? You got it already.
retriever.invoke("What's a movie that release after feb 1995?")

In [ ]:
# Don't know what your exact filter should be? Use string pattern match!
retriever.invoke("What's a movie whose name is like Andrei?")

In [ ]:
# Contain works for lists: so you can match a list with contain comparator!
retriever.invoke("What's a movie who has genres science fiction and adventure?")

## Filter k

我们还可以使用 self query retriever 来指定 `k`：要获取的文档数量。

我们可以通过向构造函数传递 `enable_limit=True` 来实现这一点。

In [ ]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [ ]:
# This example only specifies a relevant query
retriever.invoke("what are two movies about dinosaurs")